In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
#root = '/home/odroid/Downloads/experiment/'
root = 'C:/Users/SIU856536670/OneDrive - Southern Illinois University/Documents/Summer 2024/BSN 2024/experiment/'
# Load the UCI HAR dataset
def load_data(root):
    train_X_path = root + 'UCI HAR Dataset/train/X_train.txt'
    train_y_path = root + 'UCI HAR Dataset/train/y_train.txt'
    test_X_path = root + 'UCI HAR Dataset/test/X_test.txt'
    test_y_path = root + 'UCI HAR Dataset/test/y_test.txt'

    X_train = pd.read_csv(train_X_path, delim_whitespace=True, header=None).values
    y_train = pd.read_csv(train_y_path, delim_whitespace=True, header=None).values.ravel()
    X_test = pd.read_csv(test_X_path, delim_whitespace=True, header=None).values
    y_test = pd.read_csv(test_y_path, delim_whitespace=True, header=None).values.ravel()

    # Convert labels to zero-indexed
    y_train -= 1
    y_test -= 1

    return X_train, y_train, X_test, y_test


X_train, y_train, X_test, y_test = load_data(root)

# Reshape data to 3D (num_samples, seq_length, input_size)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Convert labels to categorical
y_train = to_categorical(y_train, num_classes=6)
y_test = to_categorical(y_test, num_classes=6)


def create_simple_lstm_model(input_shape, num_classes, hidden_size=64, num_layers=1):
    model = models.Sequential()
    for _ in range(num_layers):
        model.add(layers.LSTM(hidden_size, return_sequences=True if _ < num_layers - 1 else False, input_shape=input_shape))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

input_shape = (X_train.shape[1], X_train.shape[2])  # Shape: (seq_length, input_size)
num_classes = 6

model = create_simple_lstm_model(input_shape, num_classes)

In [ ]:
#%pip install codecarbon==2.5.0

In [ ]:
from codecarbon import EmissionsTracker
tracker = EmissionsTracker(project_name="IEEE_ICPS", measure_power_secs=1000)
#tracker.start()

#tracker.start()
#_ = tracker.stop()

In [ ]:
X_train, y_train, X_test, y_test = load_data(root)

# Reshape data to 3D (num_samples, seq_length, input_size)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Convert labels to categorical
y_train = to_categorical(y_train, num_classes=6)
y_test = to_categorical(y_test, num_classes=6)


def create_simple_lstm_model(input_shape, num_classes, hidden_size=64, num_layers=1):
    model = models.Sequential()
    for _ in range(num_layers):
        model.add(layers.LSTM(hidden_size, return_sequences=True if _ < num_layers - 1 else False, input_shape=input_shape))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

input_shape = (X_train.shape[1], X_train.shape[2])  # Shape: (seq_length, input_size)
num_classes = 6

vanilla_model = create_simple_lstm_model(input_shape, num_classes)
#model_without_attack.summary()
vanilla_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

def fn_vanilla_model_train(epochs):
    #print("fn_vanilla_model_train:")
    history = vanilla_model.fit(X_train, y_train, epochs=epochs, batch_size=64, validation_split=0.1)
    return vanilla_model, history



In [ ]:
from tensorflow.keras.optimizers.legacy import Adam
import tensorflow as tf
X_train, y_train, X_test, y_test = load_data(root)

# Reshape data to 3D (num_samples, seq_length, input_size)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Convert labels to categorical
y_train = to_categorical(y_train, num_classes=6)
y_test = to_categorical(y_test, num_classes=6)

# Define the LSTM model
def create_simple_lstm_model(input_shape, num_classes, hidden_size=64, num_layers=1):
    model = models.Sequential()
    for _ in range(num_layers):
        model.add(layers.LSTM(hidden_size, return_sequences=True if _ < num_layers - 1 else False, input_shape=input_shape))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

input_shape = (X_train.shape[1], X_train.shape[2])  # Shape: (seq_length, input_size)
num_classes = 6

model = create_simple_lstm_model(input_shape, num_classes)
#model.summary()

# Use legacy optimizer with a lower learning rate
optimizer = Adam(learning_rate=0.0005)

# Compile the model with the standard categorical cross-entropy loss
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define the sponge objective function
def sponge_objective(y_true, y_pred):
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    energy = tf.reduce_sum(tf.pow(y_pred, 2) / (tf.pow(y_pred, 2) + sigma))
    energy_penalty = lambda_sponge * energy
    return loss - energy_penalty

# lambda_sponge = 1.0  # Hyperparameter for energy penalty
# sigma = 1e-4  # Small constant for stability

lambda_sponge = 10.0  # Increase to prioritize energy penalty
sigma = 1e-5  # Decrease for greater sensitivity

# Custom training loop
@tf.function
def train_step_with_sponge(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        predictions = model(x, training=True)
        loss = sponge_objective(y, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

def train_with_sponge(model, X_train, y_train, epochs, batch_size=64):
    dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=1024).batch(batch_size)
    accuracy_metric = tf.keras.metrics.CategoricalAccuracy(name='accuracy')
    epoch_accuracy = 0
    for epoch in range(epochs):
        epoch_loss_avg = tf.keras.metrics.Mean()
        for x, y in dataset:
            loss = train_step_with_sponge(model, optimizer, x, y)
            epoch_loss_avg.update_state(loss)
            accuracy_metric.update_state(y, model(x, training=True))
        #print(f"Epoch {epoch+1}, Loss: {epoch_loss_avg.result().numpy()}")
        epoch_accuracy = accuracy_metric.result().numpy()
        print(f"Epoch {epoch+1}, Accuracy: {epoch_accuracy:.6f}")

    return model, epoch_accuracy
#tracker.start()#_task("train_with_sponge")
# Test the model trained with sponge attack
print("Testing model trained with attack:")


def fn_sponge_model_train(epochs):
    # Train the model with sponge attack
    sponge_model, accuracy = train_with_sponge(model, X_train, y_train, epochs)
    return sponge_model, accuracy
#model_emissions = tracker.stop_task()
#_ = tracker.stop()

In [ ]:
def fn_model_without_attack_test(vanilla_model, X_merged, y_merged):
    # Test the model trained with out attack
    print("Testing model trained with out attack:")
    results = vanilla_model.evaluate(X_merged, y_merged)
    print(f'Test Accuracy with out Attack: {results[1] * 100:.4f}%')

    return results

In [ ]:
def fn_sponge_attack_test(sponge_model, X_merged, y_merged):
    # Test the model trained with sponge attack
    print("Testing model trained with sponge attack:")
    results = sponge_model.evaluate(X_merged, y_merged)
    print(f'Test Accuracy with Sponge Attack: {results[1] * 100:.4f}%')
    #model_emissions = tracker.stop_task()
    return results

In [ ]:
X_merged = np.concatenate((X_train, X_test), axis=0)
y_merged = np.concatenate((y_train, y_test), axis=0)


for i in range(0,10):
    text_arr = []
    
    epochs = (i+1) * 5
    print(f"Iteration {i+1}: Performing an action on epoch = {epochs}")
    # Create an EmissionsTracker instance with a custom output file name
    tracker = EmissionsTracker(output_file=f"emissions_data_{epochs}.csv")




    tracker.start()
    vanilla_model, history = fn_vanilla_model_train(epochs)
    #print('history')
    #print(history.history.get('val_accuracy')[-1])
    print(f"fn_vanilla_model_train: acc {history.history.get('accuracy')[-1]:.6f}")
        # Specify the text and filename
    _ = tracker.stop()
    text = f"type: fn_vanilla_model_train, Accuracy: {history.history.get('accuracy')[-1]:.6f}"
    text_arr.append(text)

    tracker.start()
    sponge_model, accuracy = fn_sponge_model_train(epochs)
    print(f"fn_sponge_model_train: acc {accuracy:.6f}")
    _ = tracker.stop()
    text = f"type: fn_sponge_model_train, Accuracy: {accuracy:.6f}"
    text_arr.append(text)

    tracker.start()
    results = fn_model_without_attack_test(vanilla_model, X_merged, y_merged)
    print(f"fn_model_without_attack_test: acc {results[1]:.6f}")
    _ = tracker.stop()
    text = f"type: fn_model_without_attack_test, Accuracy: {results[1]:.6f}"
    text_arr.append(text)

    tracker.start()
    results = fn_sponge_attack_test(sponge_model, X_merged, y_merged )
    print(f"fn_sponge_attack_test: acc {results[1]:.6f}")
    _ = tracker.stop()
    text = f"type: fn_sponge_attack_test, Accuracy: {results[1]:.6f}"
    text_arr.append(text)
    
    #del tracker
    # Open the output file in append mode and flush the contents
    with open(f"emissions_data_{epochs}.csv", "a") as f:
        f.flush()  # Ensures that all data is written to disk immediately